# Dynamic DML for Structural Nested Mean Models

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Helper imports
import numpy as np
from sklearn.linear_model import Lasso, LassoCV, LogisticRegression, LogisticRegressionCV, MultiTaskLassoCV
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV, LogisticRegressionCV

import warnings
warnings.simplefilter('ignore')

%matplotlib inline

### Real dataL Job Corps training program

In [ ]:
df = pd.read_csv('JC.csv')
df = df.rename(columns={'Unnamed: 0':'id'}).reset_index().drop('index', axis=1).set_index(['id'])
df.head()

In [ ]:
x0_cols = list(df.columns[1:29])
x1_cols = list(df.columns[29:36])
t0_cols = df.columns[[36]]
t1_cols = df.columns[[37]]
y_col = df.columns[43]

In [ ]:
plt.hist(df['educmum'])
plt.show()

In [ ]:
y = df[y_col].values
X = {0: df[x0_cols], 1: df[x1_cols], 'het': df[x0_cols]}
T = {0: df[t0_cols].values, 1: df[t1_cols].values}
m = 2

In [ ]:
cat = ['age', 'educ', 'educmum', 'educdad']
X[0] = pd.get_dummies(X[0], columns=cat)
X[0] = pd.concat([X[0], df[cat]], axis=1)
x0_cols = list(X[0].columns)

In [ ]:
true_effect_params = np.zeros((m, T[0].shape[1]))

### Simulated Data

#### 1.1 DGP

We consider a data generating process from a markovian treatment model. 

In the example bellow, $T_t\rightarrow$ treatment(s) at time $t$, $Y_t\rightarrow$outcome at time $t$, $X_t\rightarrow$ features and controls at time $t$ (the coefficients $e, f$ will pick the features and the controls).
\begin{align}
    X_t =& (\pi'X_{t-1} + 1) \cdot A\, T_{t-1} + B X_{t-1} + \epsilon_t\\
    T_t =& \gamma\, T_{t-1} + (1-\gamma) \cdot D X_t + \zeta_t\\
    Y_t =& (\sigma' X_{t} + 1) \cdot e\, T_{t} + f X_t + \eta_t
\end{align}

with $X_0, T_0 = 0$ and $\epsilon_t, \zeta_t, \eta_t \sim N(0, \sigma^2)$. Moreover, $X_t \in R^{n_x}$, $B[:, 0:s_x] \neq 0$ and $B[:, s_x:-1] = 0$, $\gamma\in [0, 1]$, $D[:, 0:s_x] \neq 0$, $D[:, s_x:-1]=0$, $f[0:s_x]\neq 0$, $f[s_x:-1]=0$.

In [ ]:
from snmm import gen_data
m = 2
n_hetero_vars = 1
y, X, T, true_effect_params, dgp = gen_data(n_periods=m, n_units=1000, n_treatments=1,
                                       n_x=5, s_x=2, s_t=2,
                                       sigma_x=.8, sigma_t=.3, sigma_y=.1,
                                       conf_str=2,
                                       hetero_strenth=2.0, n_hetero_vars=n_hetero_vars,
                                       autoreg=1.0, gamma=.2,
                                       instance_seed=0, sample_seed=0)
X['het'] = X[0].copy()

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.hist(y)
plt.subplot(1, 3, 2)
plt.hist(T[0][:, 0], alpha=.4)
plt.hist(T[1][:, 0], alpha=.4)
plt.subplot(1, 3, 3)
plt.hist(X[0]['x0'], alpha=.4)
plt.hist(X[1]['x0'], alpha=.4)
plt.show()

In [ ]:
true_effect_params

# Analysis

In [ ]:
from snmm import get_linear_model_reg, get_linear_multimodel_reg
from snmm import get_model_reg, get_multimodel_reg
from snmm import get_poly_model_reg, get_poly_multimodel_reg
from blip import BlipSpec, SimpleHeteroBlipSpec, SimpleBlipSpec, true_param_parse

### Define Model Parameters

In [ ]:
# model_reg_fn = lambda X, y: get_model_reg(X, y, degrees=[1])
# multimodel_reg_fn = lambda X, y: get_multimodel_reg(X, y, degrees=[1])
# model_reg_fn = get_linear_model_reg
# multimodel_reg_fn = get_linear_multimodel_reg
model_reg_fn = lambda X, y: get_poly_model_reg(X, y, degree=1, interaction_only=True)
multimodel_reg_fn = lambda X, y: get_poly_multimodel_reg(X, y, degree=1, interaction_only=True)

In [ ]:
bs = SimpleHeteroBlipSpec(2).fit(X, T)
phi = bs.phi
phi_names = bs.phi_names

true_quantities = true_param_parse(X, T, true_effect_params, n_hetero_vars, m, phi, phi_names)
true_params, true_params_sel, true_policy, true_policy_delta, true_opt_policy, true_opt_policy_delta = true_quantities

def pi(t, X, T):
    return np.ones(T[t].shape)

In [ ]:
true_policy

In [ ]:
true_opt_policy

### Estimate High-Dimensional Linear Blip Model

In [ ]:
from snmm import SNMMDynamicDML
from sklearn.linear_model import LassoCV

est = SNMMDynamicDML(m=m, phi=phi, phi_names_fn=phi_names,
                     model_reg_fn=model_reg_fn,
                     model_final_fn=lambda: LassoCV())

In [ ]:
est.fit(X, T, y, pi)

In [ ]:
print(est.policy_value_)
print(true_policy)
print(est.policy_delta_simple_)
print(true_policy_delta)

In [ ]:
sig = {}
for t in range(m):
    print(f'Period {t} effects {true_effect_params[t]}')
    with pd.option_context("precision", 3):
        sig[t] = np.abs(est.psi_[t]) > 0.01
        sig[t][:T[t].shape[1]] = True
        display(est.param_summary(t, coef_thr=0.01).summary_frame())

### Post Selection Inference (not unbiased): Low Dimensional Blip Model

In [ ]:
def phi_sub(t, X, T, Tt):
    return phi(t, X, T, Tt)[:, sig[t]]

def phi_names_sub(t):
    return np.array(phi_names(t))[sig[t]]

In [ ]:
from sklearn.linear_model import LinearRegression

est_sub = SNMMDynamicDML(m=m, phi=phi_sub, phi_names_fn=phi_names_sub,
                         model_reg_fn=lambda X, y: get_model_reg(X, y, degrees=[1, 2]),
                         model_final_fn=lambda: LinearRegression(),
                         verbose=1)

In [ ]:
est_sub.fit(X, T, y, pi)

In [ ]:
print(est_sub.policy_value_)
print(true_policy)

In [ ]:
for t in range(m):
    print(f'Period {t} effects {true_effect_params[t]}')
    with pd.option_context("precision", 3):
        display(est_sub.param_summary(t).summary_frame(alpha=0.01))

### Policy Delta compared to all zero

For simple phi, where the structural parameters don't change dependent on the target, we can do sth very simple

In [ ]:
print(est_sub.policy_delta_simple_)
print(true_policy_delta)

### For complex phi we need to re-run the estimation for base

In [ ]:
est_sub.fit_base()

In [ ]:
deltapi, deltapierr = est_sub.policy_delta_complex()

In [ ]:
print(deltapi, deltapierr)

### Optimal Dynamic Policy

In [ ]:
est_sub.fit_opt(X, T, y, beta=2)

In [ ]:
print(est_sub.opt_policy_delta_simple_)
print(true_opt_policy_delta)

In [ ]:
print(est_sub.opt_policy_delta_complex())

In [ ]:
for t in range(m):
    print(f'Period {t} effects {true_effect_params[t]}')
    with pd.option_context("precision", 3):
        display(est_sub.opt_param_summary(t).summary_frame())

# Non-Parametric Heterogeneity


In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
# Helper imports
import numpy as np
from sklearn.linear_model import Lasso, LassoCV, LogisticRegression, LogisticRegressionCV, MultiTaskLassoCV
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV, LogisticRegressionCV
from snmm import get_linear_model_reg, get_linear_multimodel_reg
from snmm import get_model_reg, get_multimodel_reg
from snmm import get_poly_model_reg, get_poly_multimodel_reg
from snmm import gen_data
from econml.utilities import cross_product

import warnings
warnings.simplefilter('ignore')

%matplotlib inline

### Simulated Data

#### 1.1 DGP

We consider a data generating process from a markovian treatment model. 

In the example bellow, $T_t\rightarrow$ treatment(s) at time $t$, $Y_t\rightarrow$outcome at time $t$, $X_t\rightarrow$ features and controls at time $t$ (the coefficients $e, f$ will pick the features and the controls).
\begin{align}
    X_t =& (F(\pi'X_{t-1}) + 1) \cdot A\, T_{t-1} + B X_{t-1} + \epsilon_t\\
    T_t =& \gamma\, T_{t-1} + (1-\gamma) \cdot D X_t + \zeta_t\\
    Y_t =& (F(\sigma' X_{t}) + 1) \cdot e\, T_{t} + f X_t + \eta_t
\end{align}

with $X_0, T_0 = 0$ and $\epsilon_t, \zeta_t, \eta_t \sim N(0, \sigma^2)$. Moreover, $X_t \in R^{n_x}$, $B[:, 0:s_x] \neq 0$ and $B[:, s_x:-1] = 0$, $\gamma\in [0, 1]$, $D[:, 0:s_x] \neq 0$, $D[:, s_x:-1]=0$, $f[0:s_x]\neq 0$, $f[s_x:-1]=0$. $F$ is some arbitrary non-linear function, introducing non-linearity to the heterogeneous effect model.

In [ ]:
from experiment_hetero import FUNCTIONS
m = 2
n_hetero_vars = 1
nonlin_fn = FUNCTIONS['square']
y, X, T, true_effect_params, dgp = gen_data(n_periods=m, n_units=10000, n_treatments=1,
                                       n_x=10, s_x=2, s_t=2,
                                       sigma_x=.8, sigma_t=.3, sigma_y=.1,
                                       conf_str=2,
                                       hetero_strenth=2.0, n_hetero_vars=n_hetero_vars,
                                       autoreg=1.0, gamma=.2, nonlin_fn=nonlin_fn,
                                       instance_seed=0, sample_seed=0)
X['het'] = X[0].copy()

In [ ]:
plt.figure(figsize=(15, 5)) 
plt.subplot(1, 3, 1)
plt.hist(y)
plt.subplot(1, 3, 2)
plt.hist(T[0][:, 0], alpha=.4)
plt.hist(T[1][:, 0], alpha=.4)
plt.subplot(1, 3, 3)
plt.hist(X[0]['x0'], alpha=.4)
plt.hist(X[1]['x0'], alpha=.4)
plt.show()

In [ ]:
# model_reg_fn = lambda X, y: get_model_reg(X, y, degrees=[1])
# multimodel_reg_fn = lambda X, y: get_multimodel_reg(X, y, degrees=[1])
# model_reg_fn = get_linear_model_reg
# multimodel_reg_fn = get_linear_multimodel_reg
model_reg_fn = lambda X, y: get_poly_model_reg(X, y, degree=1, interaction_only=False)
multimodel_reg_fn = lambda X, y: get_poly_multimodel_reg(X, y, degree=1, interaction_only=True)

In [ ]:
from blip import SimpleBlipSpec

bs = SimpleBlipSpec().fit(X, T)
phi = bs.phi
phi_names = bs.phi_names

def pi(t, X, T):
    return np.ones(T[t].shape)

#### We define multiple final heterogeneous dynamic effect models

Causal forests fit a forest for $\theta(X)$, minimizing $E[(y - \theta(X)'T)^2].

For uni-variate blip model feature maps, any ML model that accepts sample weights can be used, since we can re-write it as: $E[T^2 (y/T - theta(X))^2]$, turning the problem into weighted square loss minimization. We provide the `WeightedModelFinal` wrapper that performs this transformation and wraps any ML model.

For linear models, i.e. $\theta(X) = \theta'\psi(X)$, then we can re-write this as a simple linear regression problem over the cross product of the blip feature map $\phi$ and the heterogeneous effect feature map $\psi$. We provide the `LinearModelFinal` wrapper that performs this transormations and wraps any linear model and heterogeneous effect feature map function.

In [ ]:
from hetero_utils import WeightedModelFinal, LinearModelFinal, Ensemble, GCV
from econml.grf import CausalForest
from sklearn.linear_model import LinearRegression
from econml.sklearn_extensions.linear_model import StatsModelsLinearRegression
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

cf_gen = lambda ms, md, mvl, fr: lambda: CausalForest(n_estimators=1000,
                                                    max_depth=md,
                                                    min_samples_leaf=ms,
                                                    min_balancedness_tol=0.45,
                                                    max_samples=fr,
                                                    inference=False,
                                                    min_var_fraction_leaf=mvl,
                                                    min_var_leaf_on_val=True,
                                                    random_state=123)
linear_gen = lambda: LinearModelFinal(StatsModelsLinearRegression(fit_intercept=False),
                                    lambda x: x)
lasso_gen = lambda: LinearModelFinal(LassoCV(fit_intercept=False, random_state=123),
                                    lambda x: x)
polylasso_gen = lambda: LinearModelFinal(LassoCV(fit_intercept=False, random_state=123),
                                         lambda x: Pipeline([('poly', PolynomialFeatures(degree=2, include_bias=False)),
                                                             ('sc', StandardScaler())]).fit_transform(x))
rf_gen = lambda ms, md: lambda: WeightedModelFinal(RandomForestRegressor(n_estimators=100,
                                                                min_samples_leaf=ms,
                                                                max_depth=md,
                                                                random_state=123))
lgbm_gen = lambda lr, md: lambda: WeightedModelFinal(lgb.LGBMRegressor(num_leaves=32, min_child_samples=40,
                                                                learning_rate=lr,
                                                                max_depth=md,
                                                                random_state=123))

We define model generators for many configurations of the hyperparams of each type of model, each entry in `model_gens` is a function that returns an un-fitted model.

In [ ]:
model_gens = [(f'cf{it}', cf_gen(ms, md, mvl, fr))
              for it, (ms, md, mvl, fr) in enumerate([(40, 3, None, .45),
                                                      (40, 3, None, .8),
                                                      (40, 5, None, .8),
                                                      (40, 5, 0.01, .8)])] 
model_gens += [('ols', linear_gen), ('lassocv', lasso_gen), ('2dlassocv', polylasso_gen)]
model_gens += [(f'rf{it}', rf_gen(ms, md))
               for it, (ms, md) in enumerate([(40, 3), (40, 5)])]
model_gens += [(f'lgbm{it}', lgbm_gen(lr, md))
               for it, (lr, md) in enumerate([(0.01, 1), (0.1, 1), (0.01, 3), (0.1, 3)])]

The `GCV` estimator is an estimator that supports the `fit(X, T, y)` interface and performs cross-validation and model selection among all the models in the `model_gens` list. If `ensebmle=True`, it performs soft-max ensembling. If `ensemble=False` it uses the model with the best score.

In [ ]:
gcv_gen = lambda: GCV(model_gens=model_gens, ensemble=False)

In [ ]:
from snmm import HeteroSNMMDynamicDML

het_est = HeteroSNMMDynamicDML(m=m, phi=phi, phi_names_fn=phi_names,
                               model_reg_fn=lambda X, y: get_model_reg(X, y, degrees=[1, 2]),
                               model_final_fn=gcv_gen)

In [ ]:
het_est.fit(X, T, y, pi)

In [ ]:
# model that was selected for first period hetero effect
het_est.models_[0]

In [ ]:
# model that was selected for second period hetero effect
het_est.models_[1]

In [ ]:
het_est.models_[1].scores_

In [ ]:
print(het_est.policy_value_)

In [ ]:
print(het_est.policy_delta_simple_)

In [ ]:
import seaborn as sns
if hasattr(het_est.models_[0], 'feature_importances_'):
    for t in range(m):
        impdf = het_est.feature_importances_(t)
        plt.figure(figsize=(5, 5))
        sns.barplot(y=impdf['name'], x=impdf['importance'])
        plt.show()

In [ ]:
import shap
if hasattr(het_est.models_[0], 'feature_importances_'):
    for t in range(m):
        exp = shap.Explainer(het_est.models_[t])
        shap_values = exp.shap_values(X['het'])
        shap.summary_plot(shap_values, X['het'])

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.scatter(X['het'].values[:, 0], het_est.dynamic_effects(X['het'])[0])
plt.scatter(X['het'].values[:, 0],  dgp.hetero_effect_fn(0, X['het'].values[:, :n_hetero_vars]))
plt.subplot(1, 2, 2)
plt.scatter(X['het'].values[:, 0], het_est.dynamic_effects(X['het'])[1][:, 0])
plt.scatter(X['het'].values[:, 0],  dgp.hetero_effect_fn(1, X['het'].values[:, :n_hetero_vars]))
plt.show()

In [ ]:
for t in range(m):
    error = het_est.dynamic_effects(X['het'])[t][:, 0]
    error = error - dgp.hetero_effect_fn(t, X['het'].values[:, :n_hetero_vars]).flatten()
    print(np.sqrt(np.mean(error**2)))

We now refit the final model, but with ensembling enabled

In [ ]:
het_est.model_final_fn = lambda: GCV(model_gens=model_gens, ensemble=True, beta=1000)
het_est.fit_final()

In [ ]:
het_est.models_[0]

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.scatter(X['het'].values[:, 0], het_est.dynamic_effects(X['het'])[0])
plt.scatter(X['het'].values[:, 0],  dgp.hetero_effect_fn(0, X['het'].values[:, :n_hetero_vars]))
plt.subplot(1, 2, 2)
plt.scatter(X['het'].values[:, 0], het_est.dynamic_effects(X['het'])[1][:, 0])
plt.scatter(X['het'].values[:, 0],  dgp.hetero_effect_fn(1, X['het'].values[:, :n_hetero_vars]))
plt.show()

In [ ]:
for t in range(m):
    error = het_est.dynamic_effects(X['het'])[t][:, 0]
    error = error - dgp.hetero_effect_fn(t, X['het'].values[:, :n_hetero_vars]).flatten()
    print(np.sqrt(np.mean(error**2)))

We now test how each individual model would have performed as a final model if we were to use just that model

In [ ]:
for name, mgen in model_gens:
    print(name, mgen())
    het_est.model_final_fn = mgen
    het_est.fit_final()
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.scatter(X['het'].values[:, 0], het_est.dynamic_effects(X['het'])[0])
    plt.scatter(X['het'].values[:, 0],  dgp.hetero_effect_fn(0, X['het'].values[:, :n_hetero_vars]))
    plt.subplot(1, 2, 2)
    plt.scatter(X['het'].values[:, 0], het_est.dynamic_effects(X['het'])[1][:, 0])
    plt.scatter(X['het'].values[:, 0],  dgp.hetero_effect_fn(1, X['het'].values[:, :n_hetero_vars]))
    plt.show()

    for t in range(m):
        error = het_est.dynamic_effects(X['het'].values)[t][:, 0] 
        error = error - dgp.hetero_effect_fn(t, X['het'].values[:, :n_hetero_vars]).flatten()
        print(np.sqrt(np.mean(error**2)))

Reverting final model to the ensemble one.

In [ ]:
het_est.model_final_fn = lambda: GCV(model_gens=model_gens, ensemble=True, beta=1000)
het_est.fit_final()

### Fit value of baseline policy, for delta estimation

In [ ]:
het_est.fit_base()

In [ ]:
print(het_est.policy_delta_complex())

### Optimal Dynamic Policy

In [ ]:
het_est.fit_opt(X, T, y)

In [ ]:
het_est.pi_star(1, X, T)[:10]

In [ ]:
if hasattr(het_est.models_[0], 'linear_model'):
    for t in range(m):
        print(f'Period {t} effects {true_effect_params[t]}')
        display(het_est.opt_param_summary(t).summary_frame())

In [ ]:
import seaborn as sns
if hasattr(het_est.opt_models_[0], 'feature_importances_'):
    for t in range(m):
        impdf = het_est.opt_feature_importances_(t)
        plt.figure(figsize=(5, 5))
        sns.barplot(y=impdf['name'], x=impdf['importance'])
        plt.show()

In [ ]:
import shap
if hasattr(het_est.opt_models_[0], 'feature_importances_'):
    for t in range(m):
        exp = shap.Explainer(het_est.opt_models_[t])
        shap_values = exp.shap_values(X['het'])
        shap.summary_plot(shap_values, X['het'])

In [ ]:
print(het_est.opt_policy_value_)

In [ ]:
print(het_est.opt_policy_delta_simple_)

In [ ]:
print(het_est.opt_policy_delta_complex())